<a href="https://colab.research.google.com/gist/Pratyush/919a79ece229b1c9ef304857cf2f42b9/kzg-polynomial-commitment-scheme-solved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KZG Polynomial Commitment

$\newcommand{\ck}{\mathsf{ck}}$
$\newcommand{\vk}{\mathsf{rk}}$
$\newcommand{\cm}{\mathsf{cm}}$
$\newcommand{\Setup}{\mathsf{Setup}}$
$\newcommand{\Commit}{\mathsf{Commit}}$
$\newcommand{\Open}{\mathsf{Open}}$
$\newcommand{\Verify}{\mathsf{Verify}}$
There are four algorithms in the KZG Polynomial Commitment Scheme:

* $\Setup(1^\lambda, d) \to (\ck, \vk)$.
* $\Commit(\ck, p) \to \cm$.
* $\Open(\ck, p, z) \to (\pi, v)$.
* $\Verify(\vk, \cm, z, v, \pi) \to b \in \{0, 1\}$

Let's construct these one-by-one.

## Setup and introduction to groups

We'll begin by installing `ark_algebra_py` as before, and will then go over the bilinear group APIs

In [ ]:
!pip install --upgrade ark_algebra_py

### Group APIs

We'll use *additive* notation for our group operations. So, instead of writing $g^x \cdot g^y$, we'll write $x \cdot G + y \cdot G$.

In [ ]:
from ark_algebra_py.ark_algebra_py import G1, G2, Pairing, GT, Scalar, Polynomial

G = G1();
a = Scalar(4);
b = Scalar(2);

# We can multiply the generator by scalar field elements
aG = a * G
bG = b * G;

print("a + b = ", aG + bG); # we can add...
print("a - b = ", aG - G); # ... subtract ...
print("a + -a = ", aG + -aG) # ... negate ...
print("a.double() = ", aG.double()) # ... and double points.

The same API works for $\mathbb{G}_2$ elements as well, so we won't cover it here. Let's focus on pairings next.

In [5]:
G = G1();
a = Scalar(4);
b = Scalar(2);

H = G2(); # The generator of G2 is usually denoted as H.

# We can multiply the generator by scalar field elements
aG = a * G
bH = b * H;

assert(Pairing.pairing(aG, bH) == Pairing.pairing(G, H)**8)
assert(Pairing.pairing(b * G, a * H) == Pairing.pairing(G, H)**8)

## KZG Setup

Recall that the setup algorithm looks like the following:


$\Setup(1^\lambda, d) \to (\ck, \vk)$:

1. Sample $\tau \gets \mathbb{F}_p$.
2. Set $\ck := (G, \tau G, \dots, \tau^d G)$.
3. Set $\vk := (G, H, \tau H)$.
4. Output $(\ck, \vk)$.

In [51]:
def setup(degree):
  tau = Scalar.rand()
  ck = [tau**i * G1() for i in range(0, degree + 1)]
  rk = (G1(), G2(), tau * G2())
  return (ck, rk)

## KZG Commit

Recall that the commit algorithm looks like the following:


$\Commit(\ck, p) \to \cm$:

1. Output $\cm := \sum_{i = 0}^d p_i \tau^i G$.


In [52]:
def commit(ck, polynomial):
  # hint: use `zip` function
  # hint: `polynomial.coefficients()` will return a list of `polynomial`'s coefficients
  cm = G1.identity()
  for (p_i, G_i) in zip(polynomial.coefficients(), ck):
    cm = cm + p_i * G_i;
  return cm

Let's benchmark how long this takes for a moderate degree polynomial.

In [ ]:
(ck, rk) = setup(2**15)

poly = Polynomial([Scalar(i) for i in range(0, 2**15 + 1)])
print(poly.degree()) # Should be 2^15

import time;
start = time.perf_counter();
cm = commit(ck, poly)
end = time.perf_counter();
print(end - start)

This seems kinda slow, no? Let's speed things up by using multiscalar multiplication via the `msm` method on $\mathbb{G}_1$ elements.

In [54]:
def commit_fast(ck, polynomial):
  cm = G1.msm(ck, polynomial.coefficients())
  return cm

Let's benchmark `commit_fast`.

In [ ]:
start = time.perf_counter();
cm = commit_fast(ck, poly)
end = time.perf_counter();
print(end - start)

## KZG Open


Recall that the opening algorithm looks like the following:


$\Open(\ck, p, z) \to (\pi, v)$:

1. Compute evaluation $v := p(z)$.
2. Compute quotient $q(X) := \frac{p(X) - v}{X - z}$.
3. Compute proof $\pi := \Commit(\ck, q)$.
4. Output $(\pi, v)$.


In [56]:
def open(ck, polynomial, point):
  # hint: `Polynomial.X()` construct the polynomial with just the `X` variable
  # hint: `Polynomial.constant(c) constructs the constant polynomial
  evaluation = polynomial.evaluate(point)
  numerator = polynomial - Polynomial.constant(evaluation)
  denom = Polynomial.X() - Polynomial.constant(point)
  (q, r) = numerator/denom
  assert(r == Polynomial.zero())
  proof = commit_fast(ck, q)
  return (proof, evaluation)

## KZG Verify


Recall that the verifier's algorithm looks like the following:


$\Verify(\vk, \cm, z, v, \pi) \to b \in \{0, 1\}$:

1. Check $e(\cm - v G, H) \overset?= e(\pi, \tau H - z H)$


In [57]:
def verify(rk, cm, point, evaluation, proof):
  (G, H, tauH) = rk
  lhs = Pairing.pairing(cm - evaluation * G, H)
  rhs = Pairing.pairing(proof, tauH - point * H)
  return lhs == rhs

## Completeness

Now let's check that our algorithms work!

In [58]:
poly = Polynomial([Scalar(i) for i in range(0, 2**15 + 1)])
cm = commit_fast(ck, poly)

point = Scalar(100)

(proof, evaluation) = open(ck, poly, point);
assert(verify(rk, cm, point, evaluation, proof))

## Soundness

Now let's check that we reject incorrect claims!

In [59]:
poly = Polynomial([Scalar(i) for i in range(0, 2**15 + 1)])
cm = commit_fast(ck, poly)

point = Scalar(100)

(proof, _) = open(ck, poly, point);
# We'll use an incorrect evaluation

evaluation = Scalar(0)
assert(verify(rk, cm, point, evaluation, proof) == False)